# Welcome to the Data Pipeline Master Notebook!

This notebook consists of three main parts:
1. **Data Scraping**
2. **Data Preprocessing**
3. **Data Splitting**

For each of these parts, dedicated Python scripts have been created that are later used for deployment with Docker.

> **WARNING:** These scripts must be executed strictly in the order listed above!


In [ ]:
# Import Packages
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import pandas as pd
import numpy as np
from io import StringIO
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Add the code directory to the Python path
import sys
import os

code_path = os.path.abspath(os.path.join('..'))
if code_path not in sys.path:
    sys.path.append(code_path)

In [ ]:
# Import the save_plot function from utils
from utils.plot_saver import save_plot

## Part 1: Data Scraping

### 1.1 Scrape the data from the .csv file saved on projects github repo

In [ ]:
# URL of the raw CSV file
url = 'https://raw.githubusercontent.com/il1a/student-performance-predictor/refs/heads/main/data/raw/original_data.csv'

# Make an HTTP GET request to fetch the raw CSV content
response = requests.get(url)

if response.status_code == 200:
    csv_data = StringIO(response.text)
    df = pd.read_csv(csv_data)
    print("Data scraped successfully! Here's the DataFrame information:")
    df.info()
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

### 1.2 Print some random data samples

In [ ]:
print("\n10 random data samples:")
print(df.sample(10))

## Part 2: Data Preprocessing

### 2.1 Check the number of missing values

In [ ]:
total_na = df.isna().sum().sum()
print("\nInitial total number of NA values in the dataset:", total_na)

### 2.2 Impute missing values

In [ ]:
#    - Numeric columns: fill with mean
#    - Categorical columns: fill with mode

# Separate numeric and categorical columns
numeric_cols = df.select_dtypes(include=[np.number]).columns
categorical_cols = df.select_dtypes(include=['object']).columns

# Impute numeric columns with mean
imputer_num = SimpleImputer(strategy='mean')
df[numeric_cols] = imputer_num.fit_transform(df[numeric_cols])

# Impute categorical columns with mode
imputer_cat = SimpleImputer(strategy='most_frequent')
df[categorical_cols] = imputer_cat.fit_transform(df[categorical_cols])

# Verify that no missing values remain
total_na_after_imputation = df.isna().sum().sum()
print("\nTotal number of NA values in the dataset after imputation:", total_na_after_imputation)

### 2.3 Convert categorical columns to numeric

In [ ]:

# Create dictionaries with mappings
yes_no_mapping = {
    'No': 0,
    'Yes': 1
}

gender_mapping = {
    'Female': 0,
    'Male': 1
}

ordinal_mappings = {
    'Low': 1,
    'Medium': 2,
    'High': 3
}

pos_neut_neg_mapping = {
    'Negative': 1,
    'Neutral': 2,
    'Positive': 3
}

dist_mapping = {
    'Near': 1,
    'Moderate': 2,
    'Far': 3
}

edu_mapping = {
    'High School': 1,
    'College': 2,
    'Postgraduate': 3
}

# Apply mappings to categorical columns
for col in ['Extracurricular_Activities', 'Internet_Access', 'Learning_Disabilities']:
    if col in df.columns:
        df[col] = df[col].map(yes_no_mapping)

for col in ['Parental_Involvement', 'Access_to_Resources', 'Motivation_Level', 'Family_Income', 'Teacher_Quality']:
    if col in df.columns:
        df[col] = df[col].map(ordinal_mappings)

if 'Gender' in df.columns:
    df['Gender'] = df['Gender'].map(gender_mapping)

if 'Peer_Influence' in df.columns:
    df['Peer_Influence'] = df['Peer_Influence'].map(pos_neut_neg_mapping)

if 'Distance_from_Home' in df.columns:
    df['Distance_from_Home'] = df['Distance_from_Home'].map(dist_mapping)

if 'Parental_Education_Level' in df.columns:
    df['Parental_Education_Level'] = df['Parental_Education_Level'].map(edu_mapping)

### 2.4 Choose only relevant variables (X features) based on Pearson correlation with target (Exam_Score)

In [ ]:
target_col = 'Exam_Score'

# Ensure all columns are numeric now for the correlation matrix
df_for_corr = df.select_dtypes(include=[np.number])

# Compute correlation matrix
corr_matrix = df_for_corr.corr()

# Look at correlation with target
corr_with_target = corr_matrix[target_col].abs().sort_values(ascending=False)
print("\nCorrelation with target (absolute values):\n", corr_with_target)

# Keep top 5 correlated features + the target
top_features = corr_with_target.index[0:6]
print("\nFinal top 5 features and target:\n", top_features)

# Only leave the selected top 5 features in the dataset
df_top = df_for_corr[top_features]

# Separate X and y
X = df_top.drop(columns=[target_col])
y = df_top[target_col]

### 2.5 Visualise the correlation matrix using a heatmap

In [ ]:
# Compute the correlation matrix with top 5 features
corr_matrix_top = df_top.corr()

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix_top, annot=True, cmap='magma', fmt=".2f")
plt.title("Correlation Matrix of Numeric Features")

# Save the current figure using the custom function
save_plot(plt.gcf(), 'correlation_matrix.png')

# Display the correlation matrix
plt.show()

### 2.6 Outlier detection and removal using IQR on the selected columns + target

In [ ]:
# Define additional function for IQR outlier removal
def remove_outliers_iqr(dataframe, columns, k=1.5):
    """
    Remove outliers from the specified columns using the IQR method.
    k=1.5 is the default multiplier.
    """
    df_out = dataframe.copy()
    for col in columns:
        Q1 = df_out[col].quantile(0.25)
        Q3 = df_out[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - k * IQR
        upper_bound = Q3 + k * IQR

        # Filter out rows outside the IQR bounds
        df_out = df_out[(df_out[col] >= lower_bound) & (df_out[col] <= upper_bound)]
    return df_out

# Combine X and y temporarily for outlier removal
df_xy = X.copy()
df_xy[target_col] = y

# Identify numeric columns for visualization
numeric_cols = df_xy.select_dtypes(include=[np.number]).columns

# Box plots before outlier removal
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_xy[numeric_cols], orient='v')
plt.title("Box plots Before Removing Outliers")
plt.tight_layout()
save_plot(plt.gcf(), 'boxplots_outliers.png')
plt.show()

# Perform outlier removal using IQR function defined earlier
df_xy_clean = remove_outliers_iqr(df_xy, numeric_cols, k=1.5)

# Box plots after outlier removal
plt.figure(figsize=(12, 6))
sns.boxplot(data=df_xy_clean[numeric_cols], orient='v')
plt.title("Box plots After Removing Outliers")
plt.tight_layout()
save_plot(plt.gcf(), 'boxplots_no_outliers.png')
plt.show()

# Update X and y after outlier removal
X = df_xy_clean.drop(columns=[target_col])
y = df_xy_clean[target_col]

print(f"\nShape before outlier removal: {df_xy.shape}")
print(f"Shape after outlier removal: {df_xy_clean.shape}")

### 2.7 Check data distributions visually using histograms before normalization

In [ ]:
plt.figure(figsize=(12, 12))
df_xy_clean.hist(bins=30, figsize=(12, 12))
plt.tight_layout()
save_plot(plt.gcf(), 'histograms_before_norm.png')
plt.show()

### 2.8 Print summary statistics before normalization

In [ ]:
print("\nSummary statistics before normalization:")
print(df_xy_clean.describe())

### 2.9 Perform normalization using the StandardScaler

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert back to DataFrame for easier handling
X = pd.DataFrame(X_scaled, columns=X.columns)

# Combine back X and y into one final dataset
final_clean_df = X.copy()
final_clean_df[target_col] = y.values

### 2.10 Check data distributions visually using histograms after normalization

In [ ]:
plt.figure(figsize=(12, 12))
final_clean_df.hist(bins=30, figsize=(12, 12))
plt.tight_layout()
save_plot(plt.gcf(), 'histograms_after_norm.png')
plt.show()

### 2.11 Print summary statistics after normalization

In [ ]:
print("\nSummary statistics after normalization:")
print(final_clean_df.describe())

### 2.12 Save the final clean dataset under data/processed

In [ ]:
processed_data_dir = os.path.join('..', '..', 'data', 'processed')
os.makedirs(processed_data_dir, exist_ok=True)

joint_dataset_path = os.path.join(processed_data_dir, 'joint_data_collection.csv')
final_clean_df.to_csv(joint_dataset_path, index=False)
print(f"\nFinal clean dataset saved to {joint_dataset_path}")

## Part 3: Data Splitting

### 3.1 Split the data (80% training, 20% test), save as CSV under data/processed

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

train_df = pd.DataFrame(X_train, columns=X.columns)
train_df[target_col] = y_train.values

test_df = pd.DataFrame(X_test, columns=X.columns)
test_df[target_col] = y_test.values

train_path = os.path.join(processed_data_dir, 'training_data.csv')
test_path = os.path.join(processed_data_dir, 'test_data.csv')

train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)

print(f"Training data saved to {train_path}")
print(f"Test data saved to {test_path}")

### 3.2 Create activation data (single data entry from test set), save as CSV under data/processed

In [ ]:
activation_df = test_df.sample(n=1, random_state=42)
activation_path = os.path.join(processed_data_dir, 'activation_data.csv')
activation_df.to_csv(activation_path, index=False)

print(f"Activation data (1 row) saved to {activation_path}")